## Batch features from existing tables using Signals

This notebook creates a new feature view using the SDK that will be computed using stream processing.

### Flow of data

```mermaid
flowchart LR
    wh[(Warehouse)]
    cron[/Materialization CRON job/]
    signals(Signals)

    wh --> cron
    cron --> signals
```

---

### Define a new data source

Creates a data source with the source Snowflake table configuration.

In [3]:
from snowplow_signals_sdk import DataSource

data_source = DataSource(
    name="ecommerce_transaction_interactions_source",
    database="SNOWPLOW_DEV1",
    schema="SIGNALS",
    table="SNOWPLOW_ECOMMERCE_TRANSACTION_INTERACTIONS_FEATURES",
    timestamp_field="UPDATED_AT",
)

### Create a feature view with the table fields

Feature views define the features inside the data sources and the entity types they relate to.

In [4]:
from snowplow_signals_sdk import FeatureView, user_entity, Field

feature_view = FeatureView(
    name="ecommerce_transaction_interactions_features",
    version=1,
    entities=[
        user_entity,
    ],
    fields=[
        Field(
            name="TOTAL_TRANSACTIONS",
            dtype="INT32",
        ),
        Field(
            name="TOTAL_REVENUE",
            dtype="INT32",
        ),
        Field(
            name="AVG_TRANSACTION_REVENUE",
            dtype="INT32",
        ),
    ],
    data_source=data_source,
)

### Applying the data source and feature view to Signals

The following block pushes the data source and feature view definition to the Signals API and makes it available for a background CRON job that incrementally materializes the data from the warehouse table to the online feature store.

In [5]:
from snowplow_signals_sdk import SignalsStore

sp_signals = SignalsStore()
sp_signals.apply([data_source, feature_view])

ApplyResponse(status='applied')

### Retrieving data

One can fetch the latest feature values for a particular user from the feature view as follow.

In [6]:
response = sp_signals.get_online_features(
    features=[feature_view],
    entity="9999999999999999999999999",
    entity_type_id="user",
)

response.to_dataframe()

,DOMAIN_USERID,TOTAL_TRANSACTIONS,AVG_TRANSACTION_REVENUE,TOTAL_REVENUE
0,9999999999999999999999999,1,108,108
